In [1]:
import zipfile

import os

import pandas as pd
import numpy as np

import datetime

import re

import nltk

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') # uses Multilingual Wordnet Data from OMW with newer Wordnet versions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/r.shahukaru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/r.shahukaru/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/r.shahukaru/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
from textblob import TextBlob # to use .correct() method to correct spellings

In [5]:
%%time

with zipfile.ZipFile('../data/txt_reviews.zip') as zip_ref:
    zip_ref.extractall('../data/')

CPU times: user 38.2 s, sys: 2min 10s, total: 2min 48s
Wall time: 6min 21s


In [6]:
files = [f'../data/txt_reviews/{x}' for x in os.listdir('../data/txt_reviews')]

In [7]:
%%time

info = []

for file in files:
    with open(file, 'r') as f:
        lines = f.readlines()
        
        info.append([each_line.split(':')[1].strip() for each_line in lines])

CPU times: user 9.56 s, sys: 1min 12s, total: 1min 21s
Wall time: 3min 24s


In [8]:
with open(files[0], 'r') as f:
    
    lines = f.readlines()
    cols = [each_line.split(':')[0] for each_line in lines]

In [9]:
df = pd.DataFrame(columns = cols)

In [10]:
df['ProductId'] = [information[0] for information in info]

df['UserId'] = [information[1] for information in info]

df['ProfileName'] = [information[2] for information in info]

df['HelpfulnessNumerator'] = [information[3] for information in info]

df['HelpfulnessDenominator'] = [information[4] for information in info]

df['Score'] = [information[5] for information in info]

df['Time'] = [information[6] for information in info]

df['ReviewSummary'] = [information[7] for information in info]

df['ReviewText'] = [information[8] for information in info]

In [11]:
df.dropna(inplace= True)
df.reset_index(inplace = True, drop= True)

In [12]:
df.shape

(568454, 9)

In [13]:
df.drop_duplicates(inplace= True)
df.shape

(568167, 9)

## Splitting into train and test

In [14]:
X = df[[x for x in df.columns if x != 'Score']]
y = df['Score']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.85, random_state = 100, stratify= y)

### Creating a function to clean X

In [16]:
def clean_X(X): 
    
    X['Review'] = X['ReviewSummary'] + " " + X['ReviewText']
    X.drop(['ProductId', 'UserId', 'ProfileName', 'Time', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'ReviewSummary', 'ReviewText'], axis = 1, inplace = True)
    
    
    return X

### Creating a function to clean y

In [17]:
def clean_y(y):
    y_clean = []
    
    for score in y:
        if score in ['1', '2']:
            y_clean.append('Negative')
        elif score == '3':
            y_clean.append('Neutral')
        else:
            y_clean.append('Positive')
            
    return pd.Series(y_clean, name= 'score')

## Creating a function to preprocess the data

In [18]:
# initialization
stemmer = PorterStemmer()

In [19]:
# initialization
lemmatizer = WordNetLemmatizer()

In [20]:
def preprocess(text, flag):
    
    text = re.sub("n't", " not", text)
    text = re.sub("\:\(", "bad", text)
    text = re.sub("\:\)", "good", text)
    
    #1. Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", text)
    
    #2. Converting to lowercase
    sentence_1 = sentence.lower()
    
    #3. Tokenization (Word-level)
    tokens = sentence_1.split()
    
    #3.1. TextBlob- Correcting the spellings
    tokens_correct_spell = [str(TextBlob(token)) for token in tokens]

    #4. Removing stopwords
    extra_stops = ['br', 'http', 'www', 'k']
    stop_words = [x for x in stopwords.words('english') if x not in ['not', 'but']] + extra_stops
    
    clean_tokens = [token for token in tokens_correct_spell if token not in stop_words]

    #5. stemming
    if(flag == 'stem'):
        clean_tokens_final = [stemmer.stem(word) for word in clean_tokens]
    else:
        clean_tokens_final = [lemmatizer.lemmatize(word) for word in clean_tokens]
    
    return pd.Series([" ".join(clean_tokens_final), len(clean_tokens_final)])

# 1. Train data

## 1.1.1. Cleaning X_train

In [21]:
%%time

X_train = clean_X(X_train)
X_train

CPU times: user 165 ms, sys: 225 ms, total: 390 ms
Wall time: 632 ms


,Review
380818,"Great bang for the ""caloric"" buck! At only 80 ..."
21338,"My Favorite I sent for the tea, because I can'..."
368711,New Love! I absolutely love these chocolate ba...
141202,Very fine Chamomile tea I have been drinking t...
180881,Wellness Simple Solutions Dog Food My dogs do ...
...,...
150122,"Sickening filler ingredients, high price I thi..."
295370,Bold Flavor I have tried many k-cups and this ...
118004,Nylabone Durable Dental Dinosaur My dogs had t...
303837,Pretty damn good....but I LIKE the Wawa/Conven...


## 1.1.2. Cleaning y_train

In [22]:
y_train = clean_y(y_train)
y_train

0         Positive
1         Positive
2         Positive
3         Positive
4         Positive
            ...   
482936    Negative
482937    Positive
482938    Negative
482939    Positive
482940    Positive
Name: score, Length: 482941, dtype: object

## 1.2 Preprocessing X_train

In [23]:
from tqdm import tqdm, tqdm_notebook

In [24]:
tqdm.pandas()

In [25]:
temp_df = X_train['Review'].progress_apply(lambda x: preprocess(x, 'stem'))
temp_df.columns = ['review_clean_text_stem', 'review_text_length_stem']

100%|█████████████████████████████████| 482941/482941 [04:53<00:00, 1646.65it/s]


In [26]:
X_train = temp_df
X_train

,review_clean_text_stem,review_text_length_stem
380818,great bang calor buck calori great bang calor ...,31
21338,favorit sent tea ca not buy smaller town move ...,12
368711,new love absolut love chocol bar come nice wra...,95
141202,fine chamomil tea drink chamomil tea past year...,17
180881,well simpl solut dog food dog well well simpl ...,17
...,...,...
150122,sicken filler ingredi high price think ingredi...,38
295370,bold flavor tri mani cup one far favorit deep ...,13
118004,nylabon durabl dental dinosaur dog chew apart ...,16
303837,pretti damn good but like wawa conveni store s...,47


## 1.3 Data Transformation

### 1.3.1 Bag of Words approach

#### Just considering revtext and not revsum

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# Initializing the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
vocab = CountVectorizer()

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.

X_train_bow = vocab.fit_transform(X_train['review_clean_text_stem'])

#### Unique Words

In [28]:
# We can look at unique words by using 'vocabulary_'

vocab.vocabulary_

{'great': 26396,
 'bang': 4712,
 'calor': 8948,
 'buck': 8076,
 'calori': 8950,
 'low': 35996,
 'sugar': 59320,
 'fat': 21746,
 'high': 28540,
 'protein': 48600,
 'tast': 60763,
 'like': 35116,
 'pepperoni': 45627,
 'stick': 58494,
 'without': 68315,
 'mayb': 37618,
 'littl': 35421,
 'dri': 18221,
 'side': 55424,
 'but': 8450,
 'not': 42057,
 'bad': 4463,
 'use': 65423,
 'workout': 68651,
 'favorit': 21826,
 'sent': 54416,
 'tea': 60946,
 'ca': 8641,
 'buy': 8553,
 'smaller': 56231,
 'town': 62917,
 'move': 40072,
 'glad': 25380,
 'back': 4392,
 'shelf': 54893,
 'new': 41389,
 'love': 35933,
 'absolut': 244,
 'chocol': 11238,
 'bar': 4759,
 'come': 12645,
 'nice': 41512,
 'wrap': 68788,
 'thin': 61807,
 'oz': 44368,
 'troubl': 63427,
 'set': 54580,
 'groov': 26708,
 'pattern': 45156,
 'make': 36695,
 'easi': 18772,
 'break': 7549,
 'triangl': 63243,
 'though': 61937,
 'prefer': 47859,
 'crack': 14065,
 'open': 43461,
 'eat': 18810,
 'tini': 62280,
 'piec': 46382,
 'sweet': 60038,
 'mil

In [29]:
print("Total unique words", len(vocab.vocabulary_))

print("Type of train features:", type(X_train_bow))

print("Shape of input data:", X_train_bow.shape)

Total unique words 70209
Type of train features: <class 'scipy.sparse._csr.csr_matrix'>
Shape of input data: (482941, 70209)


## 2.1.1. Cleaning X_test

In [30]:
%%time

X_test = clean_X(X_test)
X_test

CPU times: user 47.6 ms, sys: 179 ms, total: 226 ms
Wall time: 259 ms


,Review
529445,One of the best breakfast teas Yorkshire Gold ...
271152,Grilled is the best! My cat loves the Fancy Fe...
415447,"Wolfgang Puck Coffee, Vienna Coffee House grea..."
333254,Everything a Cola should be This is the best C...
71360,Like Something You might find for sale at a fo...
...,...
152671,Excellent Oatmeal Bar These Bars are the best ...
435958,Almond Anise Biscotti Celiac so this is a real...
170682,they just dont like em only 1 of my 2 cats hav...
391149,great snack This product is a healthy snack du...


## 2.1.2. Cleaning y_teset

In [31]:
y_test = clean_y(y_test)
y_test

0        Positive
1        Positive
2        Positive
3        Positive
4        Negative
           ...   
85221    Positive
85222    Positive
85223    Negative
85224    Positive
85225    Positive
Name: score, Length: 85226, dtype: object

## 2.2. Preprocessing X_test

In [32]:
temp_df = X_test['Review'].progress_apply(lambda x: preprocess(x, 'stem'))
temp_df.columns = ['review_clean_text_stem', 'review_text_length_stem']

100%|███████████████████████████████████| 85226/85226 [00:52<00:00, 1623.84it/s]


In [33]:
X_test = temp_df
X_test

,review_clean_text_stem,review_text_length_stem
529445,one best breakfast tea yorkshir gold one favor...,72
271152,grill best cat love fanci feast grill style fa...,44
415447,wolfgang puck coffe vienna coffe hous great ta...,22
333254,everyth cola best cola ever tast first tri dec...,108
71360,like someth might find sale four year old juic...,205
...,...,...
152671,excel oatmeal bar bar best ever not sweet grea...,22
435958,almond anis biscotti celiac real treat morn cu...,9
170682,dont like em cat shown interest mylar toy she ...,42
391149,great snack product healthi snack workday kid ...,17


## 2.3 Data Transformation

### Bag of Words

In [34]:
X_test_bow = vocab.transform(X_test['review_clean_text_stem'])

In [35]:
print("Total unique words", len(vocab.vocabulary_))

print("Type of test features:", type(X_test_bow))

Total unique words 70209
Type of test features: <class 'scipy.sparse._csr.csr_matrix'>


# 3. Model

## Random Forest

In [36]:
%%time

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_bow, y_train)

CPU times: user 30min 5s, sys: 7.02 s, total: 30min 12s
Wall time: 38min 45s


RandomForestClassifier()

In [37]:
y_test_pred = classifier.predict(X_test_bow)

In [38]:
from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(y_test, y_test_pred), end = "\n")

print(classification_report(y_test, y_test_pred))

0.8974960692746345
              precision    recall  f1-score   support

    Negative       0.95      0.61      0.75     12300
     Neutral       0.98      0.40      0.57      6392
    Positive       0.89      1.00      0.94     66534

    accuracy                           0.90     85226
   macro avg       0.94      0.67      0.75     85226
weighted avg       0.91      0.90      0.88     85226



In [39]:
import pickle

In [40]:
pickle.dump(vocab, open("bow.pkl", "wb"))

In [41]:
pickle.dump(classifier, open("model.pkl", "wb"))

---------------------------------------------------------------------------------------------------------------

In [42]:
def preprocess_str(text):   
      
    flag = 'stem'
    
    text = re.sub("n't", " not", text)
    text = re.sub("\:\(", "bad", text)
    text = re.sub("\:\)", "good", text)
   
    
    #1. Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", text)
    
    #2. Converting to lowercase
    sentence_1 = sentence.lower()
    
    #3. Tokenization (Word-level)
    tokens = sentence_1.split()
    
    #3.1. TextBlob- Correcting the spellings
    tokens_correct_spell = [str(TextBlob(token)) for token in tokens]

    #4. Removing stopwords
    extra_stops = ['br', 'http', 'www', 'k']
    stop_words = [x for x in stopwords.words('english') if x not in ['not', 'but']] + extra_stops
    
    clean_tokens = [token for token in tokens_correct_spell if token not in stop_words]

    #5. stemming
    if(flag == 'stem'):
        clean_tokens_final = [stemmer.stem(word) for word in clean_tokens]
    else:
        clean_tokens_final = [lemmatizer.lemmatize(word) for word in clean_tokens]
    
    return pd.Series([" ".join(clean_tokens_final)])

In [43]:
pickle.dump(preprocess_str, open("preprocess_str.pkl", "wb"))

In [44]:
preprocess_str("Very good burger.")

0    good burger
dtype: object

In [45]:
preprocess_str("Hated the food. Wouldn't recommend it.")

0    hate food would not recommend
dtype: object

In [46]:
a = vocab.transform(preprocess_str("Very good burger."))
classifier.predict(a)[0]

'Positive'

In [47]:
b = vocab.transform(preprocess_str("bad product"))
classifier.predict(b)[0]

'Negative'

In [48]:
a = vocab.transform(preprocess_str("Very good burger."))
a

<1x70209 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [ ]:
text = str(input("Please input a review: "))

a = vocab.transform(preprocess_str(text))

print()
print(classifier.predict(a)[0])

### Word Cloud

In [ ]:
preprocess(df["ReviewText"][0], 'lemm')

In [ ]:
df_lemma = pd.concat([X["ReviewText"].progress_apply(lambda x: preprocess(x, 'lemm')), y], axis=1)
df_lemma

In [ ]:
df_lemma.columns = ["clean_text_lemma", "clean_text_lemma_length", "Score"]

In [ ]:
del df_lemma["clean_text_lemma_length"]

In [ ]:
df_lemma["Score"] = clean_y(df_lemma["Score"])
df_lemma.head()

In [ ]:
negative = df_lemma.loc[df_lemma["Score"] == "Negative" , :]

words = ' '.join(negative.iloc[:, 0])

In [ ]:
from wordcloud import WordCloud

In [ ]:
negative_wordcloud = WordCloud(stopwords=stopwords.words("english"),
                      background_color='black',
                      width=1600,
                      height=800
                     ).generate(words)

In [ ]:
import matplotlib.pyplot as plt

## Negative

In [ ]:
plt.figure(1,figsize=(30,20))
plt.imshow(negative_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
positive = df_lemma.loc[df_lemma["Score"] == "Positive" , :]

words = ' '.join(positive.iloc[:, 0])

In [ ]:
positive_wordcloud = WordCloud(stopwords=stopwords.words("english"),
                      background_color='black',
                      width=1600,
                      height=800
                     ).generate(words)

## Positive

In [ ]:
plt.figure(1,figsize=(30,20))
plt.imshow(positive_wordcloud)
plt.axis('off')
plt.show()

# The END